In [3]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [57]:
cwl_dir = '/diskmnt/Projects/Users/estorrs/pecgs-somaticwrapper/cwl'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [58]:
# Step2: Type the coommand line: perl somaticwrapper.pl --srg --sre --wgs --rdir --ref --log --q --mincovt --mincovn --minvaf --maxindsize --exonic --smg --groupname --users --step

# rdir = full path of the folder holding files for this sequence run (user must provide)

# log = full path of the folder for saving log file; usually upper folder of rdir

# srg = bam having read group or not: 1, yes and 0, no (default 1)

# sre = re-run: 1, yes and 0, no (default 0)

# wgs = 1 if it is wgs data and otherwise it is 0; If you want to output the maf for all variants, set exonic to 0

# groupname = job group name: Format; users/groupname

# users = user name for job group: Format; users/groupname

# step run this pipeline step by step. (user must provide)

# ref: the human reference:

# q: which queue for submitting job; research-hpc, ding-lab, long (default)

# mincovt: minimum coverage for tumor: default >=14

# mincovn: minimum coverage for normal: default >=8

# minvaf: minimum somatic vaf: default >=0.05

# maxindsize: default <= 100

# exonic: output exonic region: 1 Yes, 0 No, Default Yes

# smg: smg gene list that escapes the 0.05 vaf cut-off



In [59]:
# my $STRELKA_DIR2="/storage1/fs1/songcao/Active/Software/strelka-2.9.10.centos6_x86_64/bin";
# my $pindel="/storage1/fs1/songcao/Active/Software/anaconda3/bin/pindel";
# my $PINDEL_DIR="/storage1/fs1/songcao/Active/Software/anaconda3/bin";
# my $picardexe="/storage1/fs1/songcao/Active/Software/picard/picard.jar";
# my $gatk="/storage1/fs1/songcao/Active/Software/GenomeAnalysis/GenomeAnalysisTK.jar";
# my $java_dir="/storage1/fs1/songcao/Active/Software/jre1.8.0_121";
# my $java_mutect="/storage1/fs1/songcao/Active/Software/jre1.7.0_80";
# my $snpsift="/storage1/fs1/songcao/Active/Software/snpEff/20150522/SnpSift.jar";
# my $gatkexe3="/storage1/fs1/songcao/Active/Software/gatk/3.7/GenomeAnalysisTK.jar";
# my $mutect1="/storage1/fs1/songcao/Active/Software/mutect/mutect-1.1.7.jar";
# my $samtools="/storage1/fs1/songcao/Active/Software/samtools/1.2/bin";
# my $samtoolsexe="/storage1/fs1/songcao/Active/Software/samtools/1.2/bin/samtools";
# my $varscan="/storage1/fs1/songcao/Active/Software/varscan/2.3.8.ndown";
# my $bamreadcount="/storage1/fs1/songcao/Active/Software/bam-readcount/0.7.4/bam-readcount";
# my $vepannot="/storage1/fs1/songcao/Active/Database/hg38_database/vep/ensembl-tools-release-85/scripts/variant_effect_predictor/variant_effect_predictor.pl";
# my $vepcache="/storage1/fs1/songcao/Active/Database/hg38_database/vep/v85";

# my $DB_SNP_NO_CHR="/storage1/fs1/songcao/Active/Database/hg38_database/DBSNP/00-All.vcf";
# my $DB_SNP="/storage1/fs1/songcao/Active/Database/hg38_database/DBSNP/00-All.chr.vcf";

# my $DB_COSMIC="/storage1/fs1/songcao/Active/Database/hg38_database/cosmic/CosmicAllMuts.HG38.sort.chr.vcf";
# my $DB_COSMIC_NO_CHR="/storage1/fs1/songcao/Active/Database/hg38_database/cosmic/CosmicAllMuts.HG38.sort.vcf"; 

# my $DB_SNP_NO_COSMIC="/storage1/fs1/songcao/Active/Database/hg38_database/cosmic/00-All.HG38.pass.cosmic.vcf";
# my $f_ref_annot="/storage1/fs1/songcao/Active/Database/hg38_database/vep/Homo_sapiens.GRCh38.dna.primary_assembly.fa";
# my $TSL_DB="/storage1/fs1/songcao/Active/Database/tsl/wgEncodeGencodeTranscriptionSupportLevelV23.txt";
# my $h38_REF_bai=$h38_REF.".fai";
# my $f_gtf= "/storage1/fs1/songcao/Active/Database/hg38_database/GTF/Homo_sapiens.GRCh38.85.gtf";

# $run_script_path = "/usr/bin/perl ".$run_script_path."/";



In [60]:
# my $STRELKA_DIR2="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/strelka-2.9.10.centos6_x86_64/bin";
# my $pindel="pindel";
# my $PINDEL_DIR="/storage1/fs1/songcao/Active/Software/anaconda3/bin";sssssssssssss
# my $picardexe="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/picard/picard.jar";
# my $gatk="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/GenomeAnalysis/GenomeAnalysisTK.jar";
# my $java_dir="//storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/jre1.8.0_121";
# my $java_mutect="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/jre1.7.0_80";
# my $snpsift="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/snpEff/20150522/SnpSift.jar";
# my $gatkexe3="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/gatk/3.7/GenomeAnalysisTK.jar";
# my $mutect1="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/mutect/mutect-1.1.7.jar";
# my $samtools="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/samtools/1.2/bin";
# my $samtoolsexe="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/samtools/1.2/bin/samtools";
# my $varscan="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/varscan/2.3.8.ndown";
# my $bamreadcount="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/software/bam-readcount/0.7.4/bam-readcount";
# my $vepannot="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/vep/ensembl-tools-release-85/scripts/variant_effect_predictor/variant_effect_predictor.pl";
# my $vepcache="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/vep/v85";

# my $DB_SNP_NO_CHR="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/DBSNP/00-All.vcf";
# my $DB_SNP="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/DBSNP/00-All.chr.vcf";

# my $DB_COSMIC="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/cosmic/CosmicAllMuts.HG38.sort.chr.vcf";
# my $DB_COSMIC_NO_CHR="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/cosmic/CosmicAllMuts.HG38.sort.vcf"; 

# my $DB_SNP_NO_COSMIC="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/cosmic/00-All.HG38.pass.cosmic.vcf";
# my $f_ref_annot="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/vep/Homo_sapiens.GRCh38.dna.primary_assembly.fa";
# my $TSL_DB="/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/tsl/wgEncodeGencodeTranscriptionSupportLevelV23.txt";
# my $h38_REF_bai=$h38_REF.".fai";
# my $f_gtf= "/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/somaticwrapper/db/hg38_database/GTF/Homo_sapiens.GRCh38.85.gtf";

# $run_script_path = "/usr/bin/perl ".$run_script_path."/";



In [61]:
template = {
    'normal_bam': {
        'class': 'File',
        'path': 'a/file/path.bam',
        'secondaryFiles': [
            {'class': 'File', 'path': f'a/file/path.bam.bai'},
        ]
    },
    'tumor_bam': {
        'class': 'File',
        'path': 'a/file/path.bam',
        'secondaryFiles': [
            {'class': 'File', 'path': f'a/file/path.bam.bai'},
        ]
    },
    'sample': 'a_string',
    'reference': {
        'class': 'File',
        'path': f'a/file/path.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': f'a/file/path.fa.amb'},
            {'class': 'File', 'path': f'a/file/path.fa.ann'},
            {'class': 'File', 'path': f'a/file/path.fa.bwt'},
            {'class': 'File', 'path': f'a/file/path.fa.fai'},
            {'class': 'File', 'path': f'a/file/path.fa.pac'},
            {'class': 'File', 'path': f'a/file/path.fa.sa'},
            {'class': 'File', 'path': f'a/file/path.dict'}
        ]
    },
    'rescue_genes': {
        'class': 'File',
        'path': 'a/file/path',
    },
}

In [62]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.somaticwrapper.yaml'), 'w'))

In [63]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
    'id': 'somaticwrapper',
    'label': 'somaticwrapper',
    'baseCommand': ['python', '/pecgs-somaticwrapper/somaticwrapper/somaticwrapper.py'],
    'inputs': [
        {
            'id': 'sample',
            'type': 'string',
            'inputBinding': {
                'position': '1'
            }
        },
        {
            'id': 'tumor_bam',
            'type': 'File',
            'secondaryFiles': [
               '.bai',
            ],
            'inputBinding': {
                'position': '2'
            }
        },
        {
            'id': 'normal_bam',
            'type': 'File',
            'secondaryFiles': [
               '.bai',
            ],
            'inputBinding': {
                'position': '3'
            }
        },
        {
            'id': 'reference',
            'type': 'File',
            'secondaryFiles': [
               '.amb',
               '.ann',
               '.bwt',
               '.fai',
               '.pac',
               '.sa',
               '^.dict'
            ],
            'inputBinding': {
                'prefix': '--reference',
                'position': '0'
            }
        },
        {
            'id': 'rescue_genes',
            'type': 'File',
            'inputBinding': {
                'prefix': '--rescue-genes',
                'position': '0'
            }
        },
        
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/usr/bin:/opt/vep/src/ensembl-vep:/opt/vep/src/var_c_code:/miniconda/envs/somaticwrapper/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--run-dir',
            'valueFrom': 'run'
        },
    ],
    'outputs': [
        {
            'id': 'dnp_annotated_maf',
            'type': 'File',
            'outputBinding': {'glob': 'run/run.dnp.annotated.maf'}
        },
        {
            'id': 'withmutect_maf',
            'type': 'File',
            'outputBinding': {'glob': 'run/run.withmutect.maf'}
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs-somaticwrapper:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 100000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [64]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'somaticwrapper.cwl'), 'w'))

In [39]:
# t = {
#     'class': 'Workflow',
#     'cwlVersion': 'v1.0',
#     'id': 'somaticwrapper_workflow',
#     'label': 'somaticwrapper_workflow',
#     'requirements': [{'class': 'ScatterFeatureRequirement'}],
#     'inputs': [
#         {
#             'id': 'sample',
#             'type': 'string',
#             'inputBinding': {
#                 'position': '1'
#             }
#         },
#         {
#             'id': 'tumor_bam',
#             'type': 'File',
#             'inputBinding': {
#                 'position': '2'
#             }
#         },
#         {
#             'id': 'normal_bam',
#             'type': 'File',
#             'inputBinding': {
#                 'position': '3'
#             }
#         },
#         {
#             'id': 'reference',
#             'type': 'File',
#             'secondaryFiles': [
#                '.fai',
#             ],
#             'inputBinding': {
#                 'prefix': '--reference',
#                 'position': '0'
#             }
#         },
#         {
#             'id': 'rescue_genes',
#             'type': 'File',
#             'inputBinding': {
#                 'prefix': '--rescue-genes',
#                 'position': '0'
#             }
#         },
#     ],
#     'outputs': [
#         {
#             'id': 'gene_level_cnv',
#             'outputSource': 'cnv/gene_level_cnv',
#             'type': 'File'
#         },
#         {
#             'id': 'arm_level_cnv',
#             'outputSource': 'cnv/arm_level_cnv',
#             'type': 'File'
#         },
#     ],
#     'steps': [
#         {
#             'id': 'somaticwrapper',
#             'in': [
# #                 {'id': 'genome', 'source': 'genome'},
# #                 {'id': 'genome_dict', 'source': 'genome_dict'},
#                 {'id': 'reference_dir', 'source': 'reference_dir'},
#                 {'id': 'target_interval_list', 'source': 'target_interval_list'},
#                 {'id': 'common_biallelic', 'source': 'common_biallelic'},
#                 {'id': 'protein_coding_gene', 'source': 'protein_coding_gene'},
#                 {'id': 'cytoband', 'source': 'cytoband'},
#                 {'id': 'pool_of_normals', 'source': 'pool_of_normals'},
#                 {'id': 'sample', 'source': 'sample'},
#                 {'id': 'normal_bam', 'source': 'stage_normal_bam/output'},
#                 {'id': 'tumor_bam', 'source': 'stage_tumor_bam/output'}],
#            'label': 'cnv',
#            'out': [{'id': 'gene_level_cnv'}, {'id': 'arm_level_cnv'}],
#            'run': './cnv.cwl'
#         },
#         {
#             'id': 'stage_normal_bam',
#             'in': [{'id': 'BAM', 'source': 'normal_bam'}],
#             'label': 'stage_normal_bam',
#             'out': [{'id': 'output'}],
#             'run': './stage_bam.cwl'
#         },
#         {
#             'id': 'stage_tumor_bam',
#             'in': [{'id': 'BAM', 'source': 'tumor_bam'}],
#             'label': 'stage_tumor_bam',
#             'out': [{'id': 'output'}],
#             'run': './stage_bam.cwl'
#         }
#     ]
# }

In [40]:
# yaml.safe_dump(t, open(os.path.join(cwl_dir, 'cnv_workflow.cwl'), 'w'))